<a href="https://colab.research.google.com/github/markwang0/GeoFlood/blob/WIP-pygeoflood-class/examples/geoflood_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install in-development branch of GeoFlood


In [1]:
%%capture

inColab = 'google.colab' in str(get_ipython())
if inColab:
    %pip install git+https://github.com/markwang0/geoflood@WIP-pygeoflood-class#egg=pygeoflood
else:
    %pip install -e ..
    pass


### Run GeoFlood operations


In [1]:
from pathlib import Path
from pygeoflood import PyGeoFlood

data_dir = Path("data")
if not data_dir.is_dir():
    data_dir.mkdir()
my_dem_path = Path(data_dir, "OC1mTest.tif")
# download DEM if it doesn't exist
if not my_dem_path.is_file():
    !curl https://utexas.box.com/shared/static/id0in7w8yn38o59sexa50aqunsr7mzk0.tif -Lso data/OC1mTest.tif

geoflood_onion = PyGeoFlood(dem_path=my_dem_path)
print(geoflood_onion)


PyGeoFlood(
    dem_path="data/OC1mTest.tif",
    project_dir="data",
)


In [2]:
# apply nonlinear filter to DEM
geoflood_onion.nonlinear_filter()

Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
DEM smoothing Quantile: 0.9
Edge Threshold Value: 0.34356024861335754
Filtered DEM written to data/OC1mTest_filtered.tif
nonlinear_filter completed in 9.0125 seconds


In [3]:
# calculate slope
geoflood_onion.slope()

 slope statistics
 angle min: 0.0
 angle max: 54.863846043721324
 mean slope: 0.13960592
 stdev slope: 0.15432632
Slope raster written to data/OC1mTest_slope.tif
slope completed in 1.2151 seconds


In [4]:
# calculate curvature
geoflood_onion.curvature()

 using geometric curvature
 curvature statistics
 non-nan curvature cell number: 16045500
 non-nan finite curvature cell number:  mean:  -0.00031314598
 standard deviation:  0.112147875
Curvature raster written to data/OC1mTest_curvature.tif
curvature completed in 1.1349 seconds


In [5]:
# fill depressions on filtered DEM
geoflood_onion.fill_depressions()

Filled DEM written to data/OC1mTest_filled.tif
fill_depressions completed in 7.4121 seconds


In [6]:
# calculate MFD flow accumulation
geoflood_onion.mfd_flow_accumulation()

MFD flow accumulation raster written to data/OC1mTest_mfd_fac.tif
mfd_flow_accumulation completed in 4.3859 seconds


In [7]:
# calculate D8 flow directions
geoflood_onion.d8_flow_direction()

D8 flow direction raster written to data/OC1mTest_d8_fdr.tif
d8_flow_direction completed in 1.6596 seconds


In [8]:
# calculate outlets
geoflood_onion.outlets()

Outlets raster written to data/OC1mTest_outlets.tif
outlets completed in 0.2105 seconds


In [9]:
# delineate basins
geoflood_onion.basins()

Basins raster written to data/OC1mTest_basins.tif
basins completed in 0.9697 seconds


In [10]:
# define skeletons
geoflood_onion.skeleton_definition(
    write_curvature_skeleton=True,
    write_flow_skeleton=True,
)

Curvature mean:  -0.00031413726
Curvature standard deviation:  0.11232527
Curvature Projection: EPSG:6343
Mean upstream flow:  2617.3135
Flow skeleton written to data/OC1mTest_flow_skeleton.tif
Curvature skeleton written to data/OC1mTest_curvature_skeleton.tif
Combined skeleton written to data/OC1mTest_combined_skeleton.tif
skeleton_definition completed in 1.0653 seconds


In [11]:
# geodesic distance (not needed for GeoFlood workflow)
geoflood_onion.geodesic_distance(write_cost_function=True)

Performing fast marching
FMM 1/32: RAM usage: 10.36/16 GB (64.8%)
FMM 2/32: RAM usage: 10.29/16 GB (64.3%)
FMM 3/32: RAM usage: 10.26/16 GB (64.1%)
FMM 4/32: RAM usage: 10.27/16 GB (64.2%)
FMM 5/32: RAM usage: 10.27/16 GB (64.2%)
FMM 6/32: RAM usage: 10.26/16 GB (64.1%)
FMM 7/32: RAM usage: 10.26/16 GB (64.1%)
FMM 8/32: RAM usage: 10.26/16 GB (64.1%)
FMM 9/32: RAM usage: 10.26/16 GB (64.1%)
FMM 10/32: RAM usage: 10.26/16 GB (64.1%)
FMM 11/32: RAM usage: 10.27/16 GB (64.2%)
FMM 12/32: RAM usage: 10.27/16 GB (64.2%)
FMM 13/32: RAM usage: 10.31/16 GB (64.4%)
FMM 14/32: RAM usage: 10.32/16 GB (64.5%)
FMM 15/32: RAM usage: 10.31/16 GB (64.5%)
FMM 16/32: RAM usage: 10.32/16 GB (64.5%)
FMM 17/32: RAM usage: 10.32/16 GB (64.5%)
FMM 18/32: RAM usage: 10.32/16 GB (64.5%)
FMM 19/32: RAM usage: 10.32/16 GB (64.5%)
FMM 20/32: RAM usage: 10.31/16 GB (64.5%)
FMM 21/32: RAM usage: 10.31/16 GB (64.4%)
FMM 22/32: RAM usage: 10.29/16 GB (64.3%)
FMM 23/32: RAM usage: 10.33/16 GB (64.6%)
FMM 24/32: RAM usa

In [12]:
# get channel heads
geoflood_onion.channel_heads()

Locating skeleton end points
Counting the number of elements of each connected component
Skeleton region size threshold: 258.7368421052632
Continuing to locate skeleton endpoints
Channel heads shapefile written to data/OC1mTest_channel_heads.shp
channel_heads completed in 2.4273 seconds


In [13]:
# set path to NFIE (NHD MR) flowline
# download here: https://www.arcgis.com/home/webmap/viewer.html?webmap=9766a82973b34f18b43dafa20c5ef535
geoflood_onion.flowline_path = Path(data_dir, "Flowline.shp")
# get flowline endpoints
geoflood_onion.endpoints()

Endpoints csv written to data/OC1mTest_endpoints.csv
endpoints completed in 0.0312 seconds


In [14]:
geoflood_onion

PyGeoFlood(
    dem_path="data/OC1mTest.tif",
    project_dir="data",
    filtered_dem_path="data/OC1mTest_filtered.tif",
    slope_path="data/OC1mTest_slope.tif",
    curvature_path="data/OC1mTest_curvature.tif",
    filled_path="data/OC1mTest_filled.tif",
    mfd_fac_path="data/OC1mTest_mfd_fac.tif",
    d8_fdr_path="data/OC1mTest_d8_fdr.tif",
    basins_path="data/OC1mTest_basins.tif",
    outlets_path="data/OC1mTest_outlets.tif",
    flow_skeleton_path="data/OC1mTest_flow_skeleton.tif",
    curvature_skeleton_path="data/OC1mTest_curvature_skeleton.tif",
    combined_skeleton_path="data/OC1mTest_combined_skeleton.tif",
    cost_function_path="data/OC1mTest_cost_function.tif",
    geodesic_distance_path="data/OC1mTest_geodesic_distance.tif",
    channel_heads_path="data/OC1mTest_channel_heads.shp",
    flowline_path="data/Flowline.shp",
    endpoints_path="data/OC1mTest_endpoints.csv",
)